In [ ]:
!pip install -U sentence-transformers
!pip install sentence-splitter
!pip install transformers
!pip install SentencePiece

In [2]:
from sentence_transformers import SentenceTransformer
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
import scipy
import numpy as np

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [5]:
import json
t = open('/content/ic.json')
data = json.load(t)
sentences = []
responses = []
for intents in data['intents']:
  sentences.extend(intents['patterns'])
  responses.extend(intents['responses'])

In [6]:
sentence_embeddings_base = model.encode(sentences)
response_embeddings_base = model.encode(responses)

In [ ]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model_pegasus = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [8]:
def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model_pegasus.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [9]:
from statistics import mode,mean

In [10]:
def average(prob_min,prob_min_index):
  avg = []
  index = []
  values = np.array(prob_min_index)
  for i in set(prob_min_index):
    a = []
    t = np.where(values == i)[0]
    for j in t:
      a.append(prob_min[j])
    avg.append(mean(a))
    index.append(i)
  return avg,index

In [11]:
def prob(queries,query_embeddings,base):
  prob_min = []
  prob_min_index = []
  for query, query_embedding in zip(queries, query_embeddings):
      distances = scipy.spatial.distance.cdist([query_embedding], base, "cosine")[0]

      prob_min.append(min(distances))
      prob_min_index.append(list(distances).index(min(distances)))
  return prob_min, prob_min_index

In [42]:
query = input('What\'s your query? : ')  
queries = get_response(query, 10) 
query_embeddings = model.encode(queries)

prob_min, prob_min_index = prob(queries,query_embeddings,sentence_embeddings_base)
avg,index = average(prob_min,prob_min_index)

if len(set([int(i*100) for i in avg])) == 1 and len([int(i*100) for i in avg]) != 1:
  print('ok')
  prob_min_resp, prob_min_index_resp = prob(queries,query_embeddings,response_embeddings_base)
  avg_resp,index_resp = average(prob_min_resp,prob_min_index_resp)
  final = index_resp[avg_resp.index(min(avg_resp))]+1
else:
  final = index[avg.index(min(avg))]+1

for intents in data['intents']:
  if intents['tag'] == 'q'+ str(final):
    print(intents['responses'][0])

What's your query? : different paper for GATE or CS paper


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


As of now there is no separate paper for CCE in GATE exam, Students will have to write the paper of computer science. But for a student who is pursuing a Computer Science related degree, learning 1-2 courses more shouldn’t pose a problem. Syllabus: (http://www.gate.iitg.ac.in)
